In [4]:
import random

class problem_generator:
    def __init__(self, n_docks:int, pallets:list, mode:str, max_heights:list, n_boxes:int, n_green:int):
        '''
        - n_docks: number of docks the problem will have
        - n_pallets: list as long as the number of docks there are. Each value in the list represents the number of pallets in the dock.
        - mode: presents how the docks are connected by the conveyor. it can be 'linear', 'circular', or even 'full'. 
        - max_heights: list as long as the number of docks there are. Each value in the list represent how high the stacks can be in that dock. 
        - n_boxes: number of boxes to be distributed along the docks. 
        - n_green: number of boxes which are green.
        '''
        self.document = ""

        # n_docks & n_cranes
        if n_docks >= 1:
            self.n_docks = n_docks
            self.n_cranes = n_docks
        else:
            raise Exception("n_docks must be greater than 0.")
        #n_pallets
        if min(pallets)> 0:
            self.n_pallets = sum(pallets)
        else:
            raise Exception("All the values in pallets must be greater than 0.")
        #n_heights
        if min(max_heights) > 0:
            self.n_heights = max(max_heights)
        else:
            raise Exception("All the values in max_heights must be greater than 0.")
        #n_boxes
        if n_boxes > 0:
            self.n_boxes = n_boxes
        else:
            raise Exception("n_boxes must be greater than 0.")
        #n_conveyors
        if mode == 'linear':
            self.n_conveyors = n_docks-1
        elif mode == "circular":
            if self.n_docks > 2:
                self.n_conveyors = n_docks
            else:
                self.n_conveyors = n_docks-1
        elif mode == "full":
            self.n_conveyors = int((n_docks*(n_docks - 1)) / 2)
        else:
            raise Exception("The mode value is invalid. It must be 'linear', 'circular' or 'full'.")
        self.n_conveyors *= 2
        #n_green
        if n_green <= n_boxes and n_green > 0:
            self.n_green = n_green
        else:
            raise Exception("Incorrect value for n_green")

        # Other variables
        self.pallets = pallets
        self.mode = mode
        self.max_heights = max_heights

        # Robustness
        if len(pallets)!=n_docks:
            raise Exception("Invalid length for pallets.")
        if len(max_heights)!=n_docks:
            raise Exception("Invalid length for n_docks.")
        if n_boxes >= sum([a*b for a,b in zip(pallets,max_heights)]):
            raise Exception("The model has more boxes that it can contain.")

    def run(self, name_problem, name_domain, path):
        self.create_problem(name_problem, name_domain)
        f = open(path+"/"+name_problem+"_dominio_"+name_domain+".pddl", "w")
        f.write(self.document)
        f.close()


    def create_problem(self, name_problem, name_domain):
        self.document += f"(define (problem {name_problem}) (:domain {name_domain})\n"
        self.create_objects()
        self.create_facts()
        self.create_goals()
        self.document += ")"

    def create_objects(self):
        '''
        Method that initializes all the possible objects
        '''
        self.document += f"(:objects\n"
        self.create_boxes()
        self.create_cranes()
        self.create_pallets()
        self.create_docks()
        self.create_conveyors()
        self.create_heights()
        self.document += ")\n\n"

    def create_boxes(self):
        '''
        Method that adds to the document the number of boxes conveyed in the initialization. 
        Example: if n_boxes=3 this method adds to the document:
        'b1 b2 b3 - box\n'
        '''
        for i in range(self.n_boxes):
            self.document += f"b{i+1} "
            self.boxes = {f"b{i+1}": None}
        self.document += "- box\n"

    def create_cranes(self):
        '''
        Method that adds to the document the number of cranes conveyed in the initialization. 
        Example: if n_cranes=3 this method adds to the document:
        'cr1 cr2 cr3 - crane\n'
        '''
        for i in range(self.n_cranes):
            self.document += f"cr{i+1} "
        self.document += "- crane\n"

    def create_pallets(self):
        '''
        Method that adds to the document the number of pallets conveyed in the initialization. 
        Example: if n_pallets=3 this method adds to the document:
        'p1 p2 p3 - pallet\n'
        '''
        for i in range(self.n_pallets):
            self.document += f"p{i+1} "
        self.document += "- pallet\n"
    
    def create_docks(self):
        '''
        Method that adds to the document the number of docks conveyed in the initialization. 
        Example: if n_docks=3 this method adds to the document:
        'd1 d2 d3 - dock\n'
        '''
        for i in range(self.n_docks):
            self.document += f"d{i+1} "
        self.document += "- dock\n"

    def create_conveyors(self):
        '''
        Method that adds to the document the number of conveyors conveyed in the initialization. 
        Example: if n_conveyors=3 this method adds to the document:
        'c1 c2 c3 - conveyor\n'
        '''
        for i in range(self.n_conveyors):
            self.document += f"c{i+1} "
        self.document += "- conveyor\n"

    def create_heights(self):
        '''
        Method that adds to the document the number of conveyors conveyed in the initialization. 
        Example: if n_heights=3 this method adds to the document:
        'h1 h2 h3 - height\n'
        '''
        for i in range(self.n_heights+1):
            self.document += f"h{i} "
        self.document += "- height\n"

    def create_facts(self):
        self.document += "(:init\n"
        self.next()
        self.max_dock_height()
        self.green()
        self.conveyors()
        self.clear()
        self.at()
        self.document += ")\n\n"

    def next(self):
        '''
        Method that adds all relationships in height:
        h0 -> h1 -> h1 -> ...
        '''
        for i in range(self.n_heights):
            self.document += f"(next h{i} h{i+1})\n"

    def max_dock_height(self):
        '''
        Method that obtains all the possible is_not_max_height facts. 
        '''
        for i in range(self.n_docks):
            for j in range(self.n_heights+1):
                if j == self.max_heights[i]:
                    continue
                self.document += f"(is_not_max_height h{j} d{i+1})\n"

    def green(self):
        '''
        Method that initializes n_green green boxes and n_boxes-n_green white boxes. 
        '''
        self.green = []
        for i in range(self.n_green):
            self.document += f"(isgreen b{i+1})\n"
            self.green += [f"b{i+1}"]
        for i in range(self.n_green, self.n_boxes):
            self.document += f"(iswhite b{i+1})\n"

    def clear(self):
        '''
        Method that writes in document the operations clear for crane and for conveyors. 
        '''
        for i in range(self.n_cranes):
            self.document += f"(clear cr{i+1})\n"
        for j in range(self.n_conveyors):
            self.document += f"(clear c{j+1})\n"

    def conveyors(self):
        '''
        Method that obtains the origin and destiny for all conveyors. 
        The operations portrayed depend on the selected mode.
        '''
        c = 1
        if self.mode == "full":
            for i in range(self.n_docks):
                for j in range(i+1, self.n_docks):
                    self.document += f"(origin c{c} d{i+1})\n"
                    self.document += f"(destiny c{c} d{j+1})\n"
                    c += 1
                    self.document += f"(origin c{c} d{j+1})\n"
                    self.document += f"(destiny c{c} d{i+1})\n"
                    c += 1
        else:
            
            for i in range(self.n_docks-1):
                self.document += f"(origin c{c} d{i+1})\n"
                self.document += f"(destiny c{c} d{i+2})\n"
                c+=1
                self.document += f"(origin c{c} d{i+2})\n"
                self.document += f"(destiny c{c} d{i+1})\n"
                c+=1
            if self.n_conveyors > c:
                self.document += f"(origin c{c} d{self.n_docks})\n"
                self.document += f"(destiny c{c} d{1})\n"
                c+=1
                self.document += f"(origin c{c} d{1})\n"
                self.document += f"(destiny c{c} d{self.n_docks})\n"

    def empty_docks(self):
        '''
        Method that initializes the pos dictionary which keys are the index for docks. 
        Its values are also dictionarys with the pallet names as keys and empty lists as values. 
        These empty lists will later be the stacks in whick we will save the boxes. 
        '''
        pallet = 1
        self.pos = {}
        for i in range(self.n_docks):
            self.pos[i] = {}
            for j in range(self.pallets[i]):
                self.pos[i][f"p{pallet}"] = []
                pallet += 1

    def fill_out_docks(self):
        '''
        Method that randomly assigns each box a pallet (and also a dock) adding it to the pos dictionary. 
        '''
        for b in range(self.n_boxes):
            repeat = True
            while repeat:
                dock = random.choice(list(self.pos.keys()))
                pallet = random.choice(list(self.pos[dock].keys()))
                if len(self.pos[dock][pallet]) < self.max_heights[dock]:
                    self.pos[dock][pallet].append(f"b{b+1}") 
                    repeat = False
        for d in self.pos.keys():
            for p in self.pos[d].keys():
                random.shuffle(self.pos[d][p])

    def at(self):
        '''
        Method that uses the pos dictionary to obtain the facts:
        - at (for cranes, pallets and boxes)
        - height (for pallets and boxes)
        - on (for boxes)
        - clear (for boxes)
        '''
        all_boxes = set([])
        not_ready = set([])
        self.empty_docks()
        self.fill_out_docks()
        pallet_at = ""
        pallet_height = ""
        boxes_at = ""
        boxes_on = ""
        boxes_height = ""
        boxes_clear = ""
        for dock in self.pos.keys():
            self.document += f"(at cr{dock+1} d{dock+1})\n"
            for pallet in self.pos[dock].keys():
                pallet_at += f"(at {pallet} d{dock+1})\n"
                pallet_height += f"(box_height h0 {pallet})\n"
                prev = pallet
                for h in range(len(self.pos[dock][pallet])):
                    box = self.pos[dock][pallet][h]
                    all_boxes.add((box, dock))
                    boxes_at += f"(at {box} d{dock+1})\n"
                    boxes_on += f"(on {box} {prev})\n"
                    boxes_height += f"(box_height h{h+1} {box})\n"
                    if h > 0:
                        if box not in self.green:
                            not_ready.add((self.pos[dock][pallet][h-1], dock))
                    if h+1 == len(self.pos[dock][pallet]):
                        boxes_clear += f"(clear {box})\n"
                    prev = box


        self.document += pallet_at
        self.document += pallet_height
        self.document += boxes_at
        self.document += boxes_on
        self.document += boxes_clear
        self.document += boxes_height
        
        for box, dock_index in list(all_boxes - not_ready):
            self.document += f"(ready {box} d{dock_index+1})\n"

    def create_goals(self, destiny_dock="d1"):
        '''
        Method that uses the destiny dock to assign the objective. 
        Example: we have green boxes b1 and b2 and d1 as destiny dock. The method would add the next text to the document:
        '(:goal (and\n(ready b1 d1)\n(ready b2 d1)\n)\n)
        '''
        self.document +="(:goal (and\n"
        for box in self.green:
            self.document += f"(ready {box} {destiny_dock})\n"
        self.document +="))\n"


In [5]:
a = set([])
a.add(1)
a

{1}

In [6]:
p = problem_generator(2, [3, 3], "linear", [4, 2], 11, 3)
p.run("unstack_simple", "dominio", ".")